In [3]:
%pip install boto3 pandas plotly requests pillow tabulate kaleido nbformat ipython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import ast

from utils import get_df_for_benchmark, performance_score, shorten_gpu_name

In [ ]:
benchmark_id = "benchmark-id-1"

# if data-1.csv exists, load it
try:
    print("Loading data from data-1.csv")
    df = pd.read_csv("./data-1.csv")
    df["timestamp"] = pd.to_datetime(df["timestamp"])
except FileNotFoundError:
    # otherwise, get the data from the API
    print("Cached CSV not found. Loading data from DynamoDB")
    df = get_df_for_benchmark(benchmark_id)
    df["gpu"] = df["gpu"].apply(shorten_gpu_name)
    df["performance_score"] = df["gpu"].apply(performance_score)
    df.to_csv("./data-1.csv", index=False)
except Exception as e:
    print("Error loading data from CSV.")
    print(e)

df.info()

In [ ]:
df.head(5)